# BAB 11: Sequence-to-Sequence Learning untuk Machine Translation

## Ringkasan

Bab ini membahas **sequence-to-sequence (seq2seq) learning**, sebuah teknik untuk memetakan sequence input dengan panjang arbitrary ke sequence output dengan panjang arbitrary. Fokus utama adalah membangun **English-to-German machine translator** menggunakan arsitektur encoder-decoder. Model encoder memproses kalimat bahasa Inggris dan menghasilkan context vector, sementara decoder menggunakan context vector tersebut untuk menghasilkan terjemahan Bahasa Jerman. Model dilatih dengan teacher forcing, dievaluasi menggunakan BLEU score, dan kemudian diadaptasi untuk inference dengan decoder rekursif.

---

## Konsep Fundamental Seq2Seq

### Encoder-Decoder Architecture

Seq2seq adalah arsitektur dua-bagian yang dirancang khusus untuk masalah di mana panjang input dan output bisa berbeda. Encoder adalah recurrent neural network yang membaca sequence sumber (misalnya, kalimat Inggris) dan menghasilkan **context vector** (juga disebut thought vector) yang merangkum informasi dari seluruh sequence input. Context vector ini adalah representasi vektor padat yang menangkap esensi dari input. Decoder kemudian menggunakan context vector sebagai keadaan awalnya untuk memproduksi sequence target (misalnya, kalimat Bahasa Jerman) word-by-word. Dengan cara ini, model dapat menghasilkan translations yang panjangnya berbeda dari input, karena tidak terikat pada length input.

### Teacher Forcing

Selama training, decoder dilatih menggunakan teknik **teacher forcing**. Bukan memberi decoder hanya context vector dan membiarkan dia menebak, kita memberikan decoder sequence target yang benar (tapi shifted by 1) sebagai input, dan melatihnya untuk memprediksi kata berikutnya. Contohnya, untuk translation "I like" → "Ich mag", decoder menerima input ["sos", "Ich"] dan dilatih untuk memprediksi output ["Ich", "mag", "eos"]. Ini jauh lebih efisien daripada mengharapkan decoder mempelajari everything dari scratch, dan convergence jauh lebih cepat.

### Bidirectional RNN

Pada encoder, kami menggunakan **Bidirectional GRU** yang membaca input kedua arah: maju (left-to-right) dan mundur (right-to-left). Ini memungkinkan encoder memahami konteks dari kedua sisi, sangat penting untuk bahasa di mana makna kata tergantung pada words di depan dan belakang. Misal, dalam "John went toward the bank on Clarence Street" vs "John went towards the bank of the river", hanya dengan membaca backward kita bisa tahu apakah "bank" merujuk ke bank keuangan atau tepi sungai. Hasil dari forward dan backward pass digabungkan (biasanya dengan concatenation) untuk menghasilkan context vector yang lebih informatif.

### SOS dan EOS Tokens

Model menggunakan dua special tokens: **SOS** (Start-Of-Sentence) dan **EOS** (End-Of-Sentence). SOS ditambahkan di awal German translation untuk menandai awal sequence, sementara EOS ditambahkan di akhir untuk menandai selesai. Tokens ini crucial untuk inference—ketika generate translation, model mulai dengan SOS token dan continues memprediksi tokens sampai menghasilkan EOS, saat itu generation berhenti.

---

## Dataset dan Preprocessing

### Machine Translation Dataset

Dataset berisi ~227,000 pasangan English-German dari tatoeba.org, tersimpan dalam format tab-separated dengan struktur `<English>\t<German>\t<Attribution>`. Kami menggunakan subset 50,000 contoh untuk mempercepat workflow. Setiap contoh adalah pasangan sentence dari dua bahasa. Dataset dibersihkan dari Unicode issues yang problematic untuk TensorFlow components.

### Data Preparation untuk Training

Data dipersiapkan melalui beberapa langkah penting. Pertama, SOS dan EOS tokens ditambahkan: "Geh." menjadi "sos Geh. eos". Kedua, data dibagi menjadi train (80%), validation (10%), dan test (10%) sets. Ketiga, karena teacher forcing, untuk setiap contoh kita membuat dua sequence: **decoder input** adalah German translation tanpa token terakhir, sementara **decoder label** adalah German translation tanpa token pertama. Misalnya untuk "Ich mag Lernen", decoder_input = "sos Ich mag" dan decoder_labels = "Ich mag Lernen eos". Ini melatih decoder untuk predict next word given previous words.

### TextVectorization Layer

TensorFlow's **TextVectorization layer** mengkonversi raw text strings ke integer token IDs. Layer ini memiliki dua tahap: **adapt** membangun vocabulary dari corpus, dan **transformation** mengkonversi new text ke sequences of IDs. Output sequence dipadding ke uniform length. Layer ini penting karena membuat model truly end-to-end—bisa menerima raw strings tanpa preprocessing external.

---

## Model Architecture

### Encoder Structure

Encoder memproses English input melalui beberapa layers. Pertama, TextVectorization layer mengkonversi string input ke token IDs. Kedua, Embedding layer mengkonversi token IDs ke dense word vectors (embedding dimension 128). Ketiga, Bidirectional GRU layer (128 units) membaca sequence tersebut, menghasilkan 256-dimensional context vector (concatenation dari forward 128D dan backward 128D). Context vector ini adalah representasi padat dari seluruh English sentence dan digunakan oleh decoder sebagai initial state.

### Decoder Structure

Decoder memiliki struktur yang lebih kompleks. Seperti encoder, ia dimulai dengan TextVectorization untuk German input dan Embedding layer untuk menghasilkan word vectors. Tetapi, GRU layer di decoder **bukan bidirectional** karena decoder harus generate translation sequentially—tidak bisa membaca forward dan backward karena output belum tersedia. Decoder GRU diinisialisasi dengan context vector dari encoder. Output dari GRU (256D) dipass ke Dense layer (512 units, ReLU) untuk representasi internal, lalu ke final Dense layer (vocabulary size, softmax) untuk menghasilkan probability distribution atas German vocabulary di setiap timestep.

### End-to-End Training Model

Model final menggabungkan encoder dan decoder dalam satu architecture yang menerima dua inputs: English sequence (untuk encoder) dan German input sequence (untuk decoder). Model dilatih dengan SparseCategoricalCrossentropy loss (karena target adalah single integer IDs, bukan one-hot vectors). Weights automatically updated via backpropagation melalui kedua components secara simultan.

---

## Evaluasi Model

### Metrics untuk Machine Translation

Model dievaluasi menggunakan tiga metrics:

**Cross-entropy Loss**: Standard multiclass loss yang mengukur mismatch antara predicted probability distribution dan true target. Lower adalah better, tapi metrik ini tidak begitu intuitive untuk translation quality.

**Accuracy**: Percentage of timesteps di mana model predict exactly the correct word. Ini strict metric karena "gato" dan "cat" (yang semantically similar) keduanya dihitung sebagai error. Jadi accuracy 50% bukan berarti translation 50% baik.

**BLEU Score**: More sophisticated metric khusus untuk translation evaluation. BLEU (Bilingual Evaluation Understudy) based pada modified precision yang tidak hanya count word matches, tapi juga mempertimbangkan n-grams. Modified precision ensures bahwa repeated words tidak dicount multiple times. BLEU juga computes precision untuk bigrams, trigrams, dll, yang memfavor translations dengan longer correct phrases intact. BLEU score dari 0.14 berarti model getting significant portions of translations correct.

---

## Inference: Dari Training ke Generation

### Problem dengan Teacher Forcing di Inference

Model trained dengan teacher forcing menerima dua inputs: English sequences dan German sequences. Selama training ini perfect karena kita tahu true German translation. Tapi di inference (real-world translation), kita hanya punya English input—German translation yang kita try generate. Kita tidak bisa gunakan true German tokens sebagai input decoder. Solusinya adalah **recursive decoder** yang generates one word at a time, feeding predicted word sebagai input untuk next timestep.

### Inference Model Architecture

Untuk inference, kami load trained model dan extract encoder dan decoder sebagai separate models. Encoder tetap sama dan dapat digunakan directly. Decoder dimodifikasi menjadi single-step decoder: bukan mengambil seluruh German sequence sebagai input, sekarang menerima single German token dan initial state sebagai input, menghasilkan single predicted token dan next state sebagai output.

### Generation Process

Proses generation adalah loop: (1) Pass English sentence ke encoder, dapatkan initial state (context vector). (2) Inisialisasi decoder input dengan SOS token dan initial state dari encoder. (3) Decoder predicts next German token dan next state. (4) Iteratively feed predicted token dan state ke decoder untuk timestep berikutnya. (5) Continue sampai decoder outputs EOS token. Output adalah complete German translation built token-by-token.

---

## Program-Program Implementasi

### Program 1: Dataset Loading dan Analysis

```python
import pandas as pd
import numpy as np

# Load data dari file tab-separated
df = pd.read_csv(os.path.join('data', 'deu.txt'), delimiter='\t', header=None)
df.columns = ["EN", "DE", "Attribution"]
df = df[["EN", "DE"]]

# Sample 50,000 contoh
n_samples = 50000
df = df.sample(n=n_samples, random_state=4321)

# Tambahkan SOS dan EOS tokens
df["DE"] = 'sos ' + df["DE"] + ' eos'

# Split menjadi train/valid/test
test_df = df.sample(n=int(n_samples/10), random_state=4321)
valid_df = df.loc[~df.index.isin(test_df.index)].sample(n=int(n_samples/10), random_state=4321)
train_df = df.loc[~(df.index.isin(test_df.index) | df.index.isin(valid_df.index))]

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(valid_df)}")
print(f"Test samples: {len(test_df)}")
```

**Penjelasan**: Program membaca dataset dari file, random sample 50,000 contoh, menambahkan special tokens ke German text, dan split menjadi 80/10/10 train/validation/test. Random sampling memastikan setiap split representative dari full dataset.

---

### Program 2: Vocabulary Analysis

```python
from collections import Counter

# Collect semua words dari training data
en_words = train_df["EN"].str.split().sum()
de_words = train_df["DE"].str.split().sum()

# Count frequencies
en_counter = Counter(en_words)
de_counter = Counter(de_words)

# Vocabulary size untuk words yang appear ≥10 times
en_vocab = sum(1 for count in en_counter.values() if count >= 10)
de_vocab = sum(1 for count in de_counter.values() if count >= 10)

# Analyze sequence lengths
en_lengths = train_df["EN"].str.split().str.len()
de_lengths = train_df["DE"].str.split().str.len()

print(f"English vocab size: {en_vocab}, Median length: {en_lengths.median()}")
print(f"German vocab size: {de_vocab}, Median length: {de_lengths.median()}")

# Set model parameters
en_seq_length = 19  # Extra padding untuk outliers
de_seq_length = 21
```

**Penjelasan**: Program analyze vocabulary size (jumlah unique words yang appear ≥10 kali) dan sequence length distribution (median, percentiles, min/max). Ini menginformasikan model hyperparameters—vocabulary size untuk embedding layers dan max sequence length untuk padding.

---

### Program 3: TextVectorization Layer Setup

```python
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def get_vectorizer(corpus, n_vocab, max_length=None, return_vocabulary=True, name=None):
    # Input layer untuk strings
    inp = tf.keras.Input(shape=(1,), dtype=tf.string, name=name+'_input')
    
    # TextVectorization layer
    vectorize_layer = TextVectorization(
        max_tokens=n_vocab+2,           # +2 untuk "" (padding) dan "[UNK]"
        output_mode='int',              # Output token IDs
        output_sequence_length=max_length
    )
    
    # Fit pada corpus untuk build vocabulary
    vectorize_layer.adapt(corpus)
    
    # Apply vectorizer
    vectorized_out = vectorize_layer(inp)
    
    # Return model dan vocabulary
    model = tf.keras.models.Model(inputs=inp, outputs=vectorized_out, name=name)
    vocab = vectorize_layer.get_vocabulary()
    
    return model, vocab

# Create vectorizers untuk English dan German
en_vectorizer, en_vocabulary = get_vectorizer(
    corpus=np.array(train_df["EN"].tolist()),
    n_vocab=en_vocab,
    max_length=en_seq_length,
    name='en_vectorizer'
)

de_vectorizer, de_vocabulary = get_vectorizer(
    corpus=np.array(train_df["DE"].tolist()),
    n_vocab=de_vocab,
    max_length=de_seq_length-1,  # -1 karena decoder input offset
    name='de_vectorizer'
)
```

**Penjelasan**: TextVectorization layer di-wrap dalam Keras Model untuk fleksibilitas. Setiap vectorizer di-fit hanya pada training data untuk avoid data leakage. Vocab size +2 untuk accommodate special padding dan [UNK] tokens yang automatically added TensorFlow.

---

### Program 4: Encoder Model

```python
def get_encoder(n_vocab, vectorizer):
    # Input layer
    inp = tf.keras.Input(shape=(1,), dtype=tf.string, name='e_input')
    
    # Vectorize
    vectorized = vectorizer(inp)
    
    # Embedding
    emb = tf.keras.layers.Embedding(
        input_dim=n_vocab+2,
        output_dim=128,
        mask_zero=True
    )(vectorized)
    
    # Bidirectional GRU—reads forward dan backward
    gru = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(128)  # Outputs 256D (128+128)
    )(emb)
    
    # Return model
    return tf.keras.models.Model(inputs=inp, outputs=gru, name='encoder')

encoder = get_encoder(en_vocab, en_vectorizer)
```

**Penjelasan**: Encoder pipeline: string → vectorized IDs → embeddings (128D) → bidirectional GRU (128 forward + 128 backward = 256D context vector). Bidirectional processing allows understanding context dari kedua directions.

---

### Program 5: Decoder dan Final Model

```python
def get_final_seq2seq_model(n_vocab, encoder, de_vectorizer):
    # Encoder input dan get context vector
    e_inp = tf.keras.Input(shape=(1,), dtype=tf.string, name='e_input_final')
    context_vector = encoder(e_inp)
    
    # Decoder input
    d_inp = tf.keras.Input(shape=(1,), dtype=tf.string, name='d_input')
    
    # Vectorize
    d_vectorized = de_vectorizer(d_inp)
    
    # Embedding
    d_emb = tf.keras.layers.Embedding(n_vocab+2, 128, mask_zero=True)(d_vectorized)
    
    # GRU dengan initial state dari encoder
    d_gru = tf.keras.layers.GRU(256, return_sequences=True)(
        d_emb, initial_state=context_vector
    )
    
    # Dense layers untuk prediction
    d_dense1 = tf.keras.layers.Dense(512, activation='relu')(d_gru)
    d_dense2 = tf.keras.layers.Dense(n_vocab+2, activation='softmax')(d_dense1)
    
    # Final model menerima English dan German, output predictions
    model = tf.keras.models.Model(
        inputs=[e_inp, d_inp],
        outputs=d_dense2,
        name='seq2seq'
    )
    
    return model

final_model = get_final_seq2seq_model(de_vocab, encoder, de_vectorizer)
final_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
```

**Penjelasan**: Final model menggabungkan encoder dan decoder. Context vector dari encoder dipass ke decoder GRU sebagai initial_state, memungkinkan decoder untuk condition generating translation pada encoder understanding dari English input.

---

### Program 6: Data Preparation untuk Training

```python
def prepare_data(train_df, valid_df, test_df):
    """Prepare datasets dalam format untuk model training"""
    data_dict = {}
    
    for label, df in zip(['train', 'valid', 'test'], [train_df, valid_df, test_df]):
        # Encoder inputs: raw English
        en_inputs = np.array(df["EN"].tolist())
        
        # Decoder inputs: German tanpa token terakhir
        de_inputs = np.array(
            df["DE"].str.rsplit(n=1, expand=True).iloc[:, 0].tolist()
        )
        
        # Decoder labels: German tanpa token pertama
        de_labels = np.array(
            df["DE"].str.split(n=1, expand=True).iloc[:, 1].tolist()
        )
        
        data_dict[label] = {
            'encoder_inputs': en_inputs,
            'decoder_inputs': de_inputs,
            'decoder_labels': de_labels
        }
    
    return data_dict

data = prepare_data(train_df, valid_df, test_df)
# data['train']['decoder_inputs'] = German tanpa last word
# data['train']['decoder_labels'] = German tanpa first word (SOS)
```

**Penjelasan**: Teacher forcing memerlukan special data formatting. Untuk setiap target sequence, kita create input (semua words except last) dan label (semua words except first). Ini melatih model predict next word given previous context.

---

### Program 7: BLEU Metric untuk Evaluasi

```python
class BLEUMetric:
    def __init__(self, vocabulary):
        self.vocab = vocabulary
        # StringLookup layer untuk convert IDs back to words
        self.id_to_word = tf.keras.layers.StringLookup(
            vocabulary=self.vocab,
            invert=True,
            num_oov_indices=0
        )
    
    def calculate_bleu_from_predictions(self, true_ids, pred_logits):
        # Convert logits to IDs
        pred_ids = tf.argmax(pred_logits, axis=-1)
        
        # Convert IDs to tokens
        pred_tokens = self.id_to_word(pred_ids)
        true_tokens = self.id_to_word(true_ids)
        
        # Clean text (remove EOS dan setelahnya)
        def clean_text(tokens):
            # Join tokens to string
            text = tf.strings.join(tf.transpose(tokens), separator=' ')
            # Remove everything after "eos"
            text = tf.strings.regex_replace(text, "eos.*", "")
            # Convert to numpy dan split
            text = text.numpy().astype(str)
            return [t.split() for t in text]
        
        pred_clean = clean_text(pred_tokens)
        true_clean = [[t.split()] for t in clean_text(true_tokens)]
        
        # Compute BLEU
        bleu, _, _, _, _, _ = compute_bleu(true_clean, pred_clean)
        return bleu

bleu_metric = BLEUMetric(de_vocabulary)
```

**Penjelasan**: BLEU metric adalah customized metric untuk translation tasks. Metric mengkonversi predicted logits menjadi token IDs, IDs menjadi words, cleanup text (remove padding dan special tokens), lalu compute BLEU score yang measures n-gram overlap dengan reference translation.

---

### Program 8: Training Loop

```python
def train_model(model, de_vectorizer, train_df, valid_df, test_df,
                epochs=5, batch_size=128):
    bleu_metric = BLEUMetric(de_vocabulary)
    data = prepare_data(train_df, valid_df, test_df)
    
    for epoch in range(epochs):
        # Shuffle training data
        train_data = data['train']
        indices = np.random.permutation(len(train_data['encoder_inputs']))
        
        for key in train_data:
            train_data[key] = train_data[key][indices]
        
        # Training batches
        n_batches = len(train_data['encoder_inputs']) // batch_size
        loss_list, acc_list, bleu_list = [], [], []
        
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            
            # Get batch
            x = [
                train_data['encoder_inputs'][start:end],
                train_data['decoder_inputs'][start:end]
            ]
            y = de_vectorizer(train_data['decoder_labels'][start:end])
            
            # Train
            model.train_on_batch(x, y)
            
            # Evaluate
            loss, acc = model.evaluate(x, y, verbose=0)
            pred_y = model.predict(x, verbose=0)
            bleu = bleu_metric.calculate_bleu_from_predictions(y, pred_y)
            
            loss_list.append(loss)
            acc_list.append(acc)
            bleu_list.append(bleu)
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Train - Loss: {np.mean(loss_list):.4f}, Acc: {np.mean(acc_list):.4f}, BLEU: {np.mean(bleu_list):.4f}")

train_model(final_model, de_vectorizer, train_df, valid_df, test_df, epochs=5)
```

**Penjelasan**: Training loop mengiterate epochs, shuffle training data setiap epoch, process batch-by-batch, train dan evaluate simultaneously. Log metrics tracked untuk monitoring training progress.

---

### Program 9: Inference Model Setup

```python
def get_inference_model(trained_model_path):
    # Load trained model
    model = tf.keras.models.load_model(trained_model_path)
    
    # Extract encoder
    en_model = model.get_layer("encoder")
    
    # Build new decoder untuk single-step inference
    d_inp = tf.keras.Input(shape=(1,), dtype=tf.string)
    d_state = tf.keras.Input(shape=(256,))  # Initial state dari encoder
    
    # Vectorize dan embed
    d_vec = model.get_layer('d_vectorizer')(d_inp)
    d_emb = model.get_layer('d_embedding')(d_vec)
    
    # GRU—return single output, not sequence
    d_gru = model.get_layer('d_gru')
    d_gru.return_sequences = False
    d_gru_out = d_gru(d_emb, initial_state=d_state)
    
    # Dense layers
    d_dense = model.get_layer('d_dense_1')(d_gru_out)
    d_out = model.get_layer('d_dense_final')(d_dense)
    
    # Decoder returns prediction dan next state
    de_model = tf.keras.models.Model(
        inputs=[d_inp, d_state],
        outputs=[d_out, d_gru_out]
    )
    
    return en_model, de_model

en_model, de_model = get_inference_model('models/seq2seq')
```

**Penjelasan**: Inference model extract weights dari trained model tapi restructure decoder untuk operate sequentially. Key change: decoder GRU set to `return_sequences=False` agar output single timestep, bukan sequence, memudahkan recursive feeding.

---

### Program 10: Translation Generation

```python
def generate_translation(en_model, de_model, de_vocabulary, english_text):
    # Get context vector
    context = en_model.predict(np.array([english_text]), verbose=0)
    
    # Initialize with SOS token
    current_token = 'sos'
    translation = []
    
    # Generate until EOS
    while current_token != 'eos':
        # Predict next token
        pred_logits, context = de_model.predict(
            [np.array([current_token]), context],
            verbose=0
        )
        
        # Get token ID dengan highest probability
        token_id = np.argmax(pred_logits[0])
        current_token = de_vocabulary[token_id]
        
        translation.append(current_token)
    
    return ' '.join(translation[:-1])  # Remove EOS token

# Example
result = generate_translation(en_model, de_model, de_vocabulary, "Hello world")
print(result)  # Output: "Hallo Welt eos"
```

**Penjelasan**: Generation process adalah autoregressive loop. Start dengan SOS token, predict next token given current token dan state, feed prediction sebagai next input, repeat sampai EOS token generated. Resulting translation adalah sequence predicted tokens joined bersama.

---

## Kesimpulan

Bab ini menunjukkan cara membangun complete end-to-end machine translation system menggunakan seq2seq architecture. Key insights: encoder-decoder design elegantly handles variable-length sequences, teacher forcing accelerates training, TextVectorization layer enables true end-to-end models yang menerima raw strings, BLEU metric lebih appropriate daripada accuracy untuk translation evaluation, dan inference requires separate model architecture yang operates recursively untuk generate translations token-by-token. Bab selanjutnya (Chapter 12) akan improve model ini dengan attention mechanism yang memungkinkan decoder fokus pada specific parts of encoder input saat generating setiap target token.
